
# Análisis Exploratorio (EDA) WiDS 2024

**Tema:** Diagnóstico temprano de cáncer de mama y variables asociadas al estado de salud y residencia de pacientes.  
**Competencia:** [WiDS Datathon 2024 — Challenge 1](https://www.kaggle.com/competitions/widsdatathon2024-challenge1)  

**Objetivo de esta actividad:** 
Análisis Exploratorio de Datos (EDA) sólido para comprender:
- tipos de variables, distribuciones, valores faltantes, inconsistencias y categorías (entre train y test),
- relaciones básicas con la variable objetivo **`DiagPeriod90`** (en `train`),
- y dejar lista una **base de preparación** para el modelado posterior.



## Contenido del Notebook
1. **importación de librerías y Configuración de rutas**  
2. **Carga y revisión general** (dimensiones, tipos, primeras filas)  
3. **Comparación de columnas entre `train` y `test`**  
4. **Valores únicos y categorías** (desalineaciones y criterios de solución)  
5. **Valores faltantes y anomalías** (nulos >30%, imputación/eliminación; detección de outliers)  
6. **Distribuciones de variables y visualizaciones**  
7. **Relaciones con `DiagPeriod90`** (conteos, comparaciones básicas)  
8. **Conclusiones y preparación para modelado** (resumen + transformaciones propuestas)  

## 1) Importación de Librerías

* Se importan las librearías necesarias para trabajar con los datos 

In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import unicodedata
import os

In [ ]:
# Ruta base del proyecto
#BASE_DIR = r"C:\Users\INIFAP-MOVIL\Documents\3 TERCER SEMESTRE\Topicos II\Trabajos\Topicos_II\Topicos_II\data"

# Rutas completas a los archivos
#TRAIN_PATH = BASE_DIR + r"\training.csv"
#TEST_PATH  = BASE_DIR + r"\test.csv"

# Carga de los DataFrames
df_training = pd.read_csv(TRAIN_PATH)
df_test = pd.read_csv(TEST_PATH)
df_training.head()
display(df_training.head())


# Revisión rápida
#print("Tamaño de training:", df_training.shape)
#print("Tamaño de test:", df_test.shape)
#display(df_training.head())
#display(df_test.head())


,patient_id,patient_race,payer_type,patient_state,patient_zip3,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,metastatic_first_novel_treatment,metastatic_first_novel_treatment_type,Region,Division,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,age_50s,age_60s,age_70s,age_over_80,male,female,married,divorced,never_married,widowed,family_size,family_dual_income,income_household_median,income_household_under_5,income_household_5_to_10,income_household_10_to_15,income_household_15_to_20,income_household_20_to_25,income_household_25_to_35,income_household_35_to_50,income_household_50_to_75,income_household_75_to_100,income_household_100_to_150,income_household_150_over,income_household_six_figure,income_individual_median,home_ownership,housing_units,home_value,rent_median,rent_burden,education_less_highschool,education_highschool,education_some_college,education_bachelors,education_graduate,education_college_or_above,education_stem_degree,labor_force_participation,unemployment_rate,self_employed,farmer,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02,DiagPeriodL90D
0,475714,NaN,MEDICAID,CA,924,84,F,NaN,C50919,Malignant neoplasm of unsp site of unspecified...,C7989,NaN,NaN,West,Pacific,31437.75000,1189.562500,30.642857,16.014286,15.542857,17.614286,14.014286,11.614286,11.557143,7.571429,4.000000,2.100000,49.857143,50.142857,36.571429,11.885714,47.114286,4.442857,3.928571,52.228571,52996.28571,3.142857,4.000000,6.157143,5.142857,6.271429,10.142857,13.300000,20.000000,12.742857,11.571429,7.528571,19.100000,24563.57143,44.585714,8674.500000,2.646343e+05,1165.000000,37.442857,33.257143,29.200000,25.914286,8.357143,3.257143,11.614286,39.557143,61.528571,8.471429,13.428571,0.000000,44.100000,13.100000,5.100000,1.485714,0.342857,27.114286,8.757143,66.685714,12.871429,22.542857,10.100000,27.814286,11.200000,3.500000,52.237210,8.650555,18.606528,1
1,349367,White,COMMERCIAL,CA,928,62,F,28.49,C50411,Malig neoplm of upper-outer quadrant of right ...,C773,NaN,NaN,West,Pacific,39121.87879,2295.939394,38.200000,11.878788,13.354545,14.230303,13.418182,13.333333,14.060606,10.248485,5.951515,3.503030,49.893939,50.106061,50.245455,9.827273,35.290909,4.651515,3.622727,61.736364,102741.63640,2.327273,1.536364,2.648485,2.178788,2.409091,5.163636,7.972727,13.936364,12.469697,19.760606,29.596970,49.357576,41287.27273,61.463636,11725.666670,6.776885e+05,2003.125000,34.753125,14.230303,19.987879,29.796970,23.739394,12.245455,35.984848,47.918182,65.230303,5.103030,15.224242,0.027273,54.030303,2.527273,20.827273,0.587879,0.300000,11.645455,10.081818,37.948485,8.957576,10.109091,8.057576,30.606061,7.018182,4.103030,42.301121,8.487175,20.113179,1
2,138632,White,COMMERCIAL,TX,760,43,F,38.09,C50112,Malignant neoplasm of central portion of left ...,C773,NaN,NaN,South,West South Central,21996.68333,626.236667,37.906667,13.028333,14.463333,12.531667,13.545000,12.860000,12.770000,11.426667,6.565000,2.811667,50.123333,49.876667,55.753333,12.330000,27.195000,4.710000,3.260667,55.801667,85984.74138,2.483333,1.305000,2.716667,2.938333,2.766667,6.763333,12.061667,15.835000,13.560000,20.875000,18.680000,39.555000,40399.03333,72.745000,7786.583333,2.377131e+05,1235.907407,29.358491,10.811667,27.038333,32.368333,19.678333,10.115000,29.793333,37.308475,66.428333,4.560000,13.722034,3.650847,75.820000,9.231667,3.618333,0.463333,0.146667,3.816667,6.898333,19.370000,11.253333,9.663333,3.356667,31.394915,15.066667,7.446667,40.108207,7.642753,14.839351,1
3,617843,White,COMMERCIAL,CA,926,45,F,NaN,C50212,Malig neoplasm of upper-inner quadrant of left...,C773,NaN,NaN,West,Pacific,32795.32558,1896.220930,42.871429,10.071429,12.135714,12.538095,12.464286,12.650000,14.847619,12.280952,8.216667,4.759524,49.066667,50.933333,52.604762,11.623810,31.142857,4.62

In [10]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import unicodedata
import os

pd.set_option('display.max_columns', 200) 
pd.set_option('display.width', 120)

# ----------------------
# RUTA BASE EN WINDOWS
# ----------------------
BASE_DIR = r"C:\Users\INIFAP-MOVIL\Documents\3 TERCER SEMESTRE\Topicos II\Trabajos\Topicos_II\Topicos_II"

TRAIN_NAME = "train.csv"
TEST_NAME  = "test.csv"

TRAIN_PATH = os.path.join(BASE_DIR, TRAIN_NAME)
TEST_PATH  = os.path.join(BASE_DIR, TEST_NAME)

print("Ruta train:", TRAIN_PATH)
print("Ruta test :", TEST_PATH)


Ruta train: C:\Users\INIFAP-MOVIL\Documents\3 TERCER SEMESTRE\Topicos II\Trabajos\Topicos_II\Topicos_II\train.csv
Ruta test : C:\Users\INIFAP-MOVIL\Documents\3 TERCER SEMESTRE\Topicos II\Trabajos\Topicos_II\Topicos_II\test.csv


In [12]:
df_training.head()

NameError: name 'df_training' is not defined


## 2) Carga y revisión general (shape, dtypes, primeras filas)
Objetivo: confirmar que los archivos existen, cargarlos y observar su estructura básica.


In [11]:

# Verificamos existencia
if not os.path.exists(TRAIN_PATH):
    raise FileNotFoundError(f"No se encontró el archivo de entrenamiento en: {TRAIN_PATH}")
if not os.path.exists(TEST_PATH):
    raise FileNotFoundError(f"No se encontró el archivo de prueba en: {TEST_PATH}")

# Carga con fallback de encoding
try:
    train = pd.read_csv(TRAIN_PATH)
    test  = pd.read_csv(TEST_PATH)
except UnicodeDecodeError:
    train = pd.read_csv(TRAIN_PATH, encoding='latin-1')
    test  = pd.read_csv(TEST_PATH,  encoding='latin-1')

print("Dimensiones train:", train.shape)
print("Dimensiones test :", test.shape)

print("\nTipos de datos en train (conteo):")
print(train.dtypes.value_counts())

print("\nTipos de datos en test (conteo):")
print(test.dtypes.value_counts())

print("\nPrimeras filas de train:")
display(train.head())

print("\nPrimeras filas de test:")
display(test.head())


FileNotFoundError: No se encontró el archivo de entrenamiento en: C:\Users\INIFAP-MOVIL\Documents\3 TERCER SEMESTRE\Topicos II\Trabajos\Topicos_II\Topicos_II\train.csv


## 3) Comparación de columnas entre `train` y `test`
- ¿Son idénticas?  
- ¿Falta alguna variable en alguno de los dos?


In [ ]:

cols_train = set(train.columns)
cols_test  = set(test.columns)

only_in_train = sorted(list(cols_train - cols_test))
only_in_test  = sorted(list(cols_test - cols_train))
in_both = sorted(list(cols_train & cols_test))

print("Columnas SOLO en train:", only_in_train)
print("Columnas SOLO en test :", only_in_test)
print("Columnas en AMBOS     :", len(in_both))

common_columns = in_both.copy()

if 'DiagPeriod90' in only_in_train:
    print("\nNota: 'DiagPeriod90' está solo en train (como es esperado para la variable objetivo).")
elif 'DiagPeriod90' in train.columns:
    print("\n'DiagPeriod90' está presente en train.")
else:
    print("\nAdvertencia: no se encontró 'DiagPeriod90' en train. Revisa el nombre exacto de la columna objetivo.")



## 4) Revisión de valores únicos y categorías
Listamos **valores únicos** y detectamos **categorías desalineadas** entre train y test. También proponemos normalización (minúsculas, sin acentos).


In [ ]:

numeric_cols = [c for c in train.columns if pd.api.types.is_numeric_dtype(train[c])]
categorical_cols = [c for c in train.columns if c not in numeric_cols]

print("Numéricas (train):", len(numeric_cols))
print("Categóricas (train):", len(categorical_cols))

def normalize_text(x):
    if pd.isna(x):
        return x
    s = str(x)
    s = unicodedata.normalize('NFKD', s)
    s = ''.join(ch for ch in s if not unicodedata.combining(ch))
    s = s.lower().strip()
    return s

mismatches = {}
for col in categorical_cols:
    ut = set(train[col].dropna().astype(str).unique())
    uv = set(test[col].dropna().astype(str).unique())
    inter = ut & uv
    diff_train = ut - uv
    diff_test  = uv - ut
    if diff_train or diff_test:
        mismatches[col] = {
            "solo_en_train": sorted(list(diff_train))[:30],
            "solo_en_test" : sorted(list(diff_test))[:30],
            "ejemplo_inter": sorted(list(inter))[:10]
        }

print(f"Columnas categóricas con categorías desalineadas: {len(mismatches)}")
for i, (col, info) in enumerate(mismatches.items()):
    if i >= 5:
        break
    print(f"\n[{col}]")
    print("  Ej. intersección:", info['ejemplo_inter'][:5])
    print("  Solo en train   :", info['solo_en_train'][:5])
    print("  Solo en test    :", info['solo_en_test'][:5])

# Versiones normalizadas (copias)
train_cat_norm = train.copy()
test_cat_norm  = test.copy()
for col in categorical_cols:
    if pd.api.types.is_object_dtype(train_cat_norm[col]) or pd.api.types.is_categorical_dtype(train_cat_norm[col]):
        train_cat_norm[col] = train_cat_norm[col].apply(normalize_text)
    if col in test_cat_norm.columns and (pd.api.types.is_object_dtype(test_cat_norm[col]) or pd.api.types.is_categorical_dtype(test_cat_norm[col])):
        test_cat_norm[col] = test_cat_norm[col].apply(normalize_text)

print("\nSe creó train_cat_norm y test_cat_norm con categorías normalizadas para posibles pruebas.")



**Sugerencias para solucionar desalineaciones:**
1. Normalizar texto (minúsculas, sin acentos).  
2. Mapear equivalencias mediante un diccionario.  
3. Asignar categoría `"unknown"` para niveles presentes en test y ausentes en train (evita errores de *encoding*).  
4. Revisar encoding si aparecen símbolos extraños.



## 5) Valores faltantes y anomalías
Calculamos porcentaje de nulos, marcamos variables con **>30%** y proponemos acciones. Luego, detectamos **outliers** con método IQR.


In [ ]:

null_pct = train.isna().mean().sort_values(ascending=False) * 100
display(null_pct.to_frame("porcentaje_nulos_%").head(30))

high_nulls = null_pct[null_pct > 30].index.tolist()
print("\nColumnas con >30% de nulos:", len(high_nulls))
print(high_nulls[:20])

for col in high_nulls:
    tipo = "numérica" if col in numeric_cols else "categórica"
    if null_pct[col] > 60:
        decision = "Posible eliminación (demasiados nulos), salvo que sea variable clave."
    else:
        decision = "Imputación sugerida: mediana (numérica) o 'Unknown' (categórica)."
    print(f"- {col} ({tipo}, {null_pct[col]:.1f}% nulos) → {decision}")


In [ ]:

outlier_report = []
for col in numeric_cols:
    series = train[col].dropna()
    if series.empty:
        continue
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    low = q1 - 1.5 * iqr
    high = q3 + 1.5 * iqr
    outliers = ((series < low) | (series > high)).sum()
    outlier_report.append((col, int(outliers), int(series.size)))

outlier_df = pd.DataFrame(outlier_report, columns=["columna", "n_outliers", "n_validos"])
outlier_df["pct_outliers_%"] = (outlier_df["n_outliers"] / outlier_df["n_validos"].replace(0, np.nan)) * 100
outlier_df = outlier_df.sort_values("pct_outliers_%", ascending=False)
display(outlier_df.head(15))



## 6) Distribuciones de variables y visualizaciones
- Usamos **matplotlib** (sin estilos/colores específicos).  
- Cada gráfico en su **propia figura**.


In [ ]:

num_to_plot = numeric_cols[:6]
for col in num_to_plot:
    plt.figure()
    train[col].dropna().hist(bins=30)
    plt.title(f"Histograma - {col} (train)")
    plt.xlabel(col)
    plt.ylabel("Frecuencia")
    plt.show()


In [ ]:

cat_to_plot = categorical_cols[:4]
for col in cat_to_plot:
    vc = train[col].astype(str).value_counts().head(10)
    plt.figure()
    vc.plot(kind="bar")
    plt.title(f"Top 10 categorías - {col} (train)")
    plt.xlabel(col)
    plt.ylabel("Frecuencia")
    plt.tight_layout()
    plt.show()



## 7) Relaciones con la variable objetivo `DiagPeriod90`
Mostramos conteos y **boxplots** para algunas variables numéricas vs `DiagPeriod90`.


In [ ]:

if 'DiagPeriod90' in train.columns:
    print("Distribución de DiagPeriod90 (train):")
    print(train['DiagPeriod90'].value_counts(dropna=False))
    
    cols_for_box = [c for c in numeric_cols if c != 'DiagPeriod90'][:4]
    for col in cols_for_box:
        tmp = train[[col, 'DiagPeriod90']].dropna()
        if tmp.empty:
            continue
        plt.figure()
        tmp.boxplot(column=col, by='DiagPeriod90')
        plt.title(f"Boxplot de {col} por DiagPeriod90 (train)")
        plt.suptitle("")
        plt.xlabel("DiagPeriod90")
        plt.ylabel(col)
        plt.show()
else:
    print("No se encontró 'DiagPeriod90' en train. Revisa el nombre exacto de la columna objetivo.")



## 8) Conclusiones y preparación para el modelado

**Resumen recomendado (edita con tus hallazgos):**
- *Columnas idénticas/diferentes:* …  
- *Variables con >30% nulos y decisión:* …  
- *Principales outliers y tratamiento:* …  
- *Variables más útiles y por qué:* …  
- *Variables que requieren codificación/limpieza:* …  
- *Transformaciones aplicadas/propuestas:* normalización de categorías, imputación (mediana/moda), codificación One-Hot, escalado en fase de modelado, winsorización/log si aplica.



## (Opcional) Pipeline mínimo de preparación (sin modelar)
Transformaciones básicas para dejar datos listos para la siguiente etapa.


In [ ]:

def normalize_text(x):
    if pd.isna(x):
        return x
    s = str(x)
    import unicodedata as _ud
    s = _ud.normalize('NFKD', s)
    s = ''.join(ch for ch in s if not _ud.combining(ch))
    s = s.lower().strip()
    return s

def simple_prepare(train_df, test_df, target_col='DiagPeriod90'):
    tr = train_df.copy()
    te = test_df.copy()
    y = None
    if target_col in tr.columns:
        y = tr[target_col]
        tr = tr.drop(columns=[target_col])
    
    num_cols = [c for c in tr.columns if pd.api.types.is_numeric_dtype(tr[c])]
    cat_cols = [c for c in tr.columns if c not in num_cols]
    
    for col in cat_cols:
        if pd.api.types.is_object_dtype(tr[col]) or pd.api.types.is_categorical_dtype(tr[col]):
            tr[col] = tr[col].apply(normalize_text)
        if col in te.columns and (pd.api.types.is_object_dtype(te[col]) or pd.api.types.is_categorical_dtype(te[col])):
            te[col] = te[col].apply(normalize_text)
    
    for col in num_cols:
        med = tr[col].median()
        tr[col] = tr[col].fillna(med)
        if col in te.columns:
            te[col] = te[col].fillna(med)
    for col in cat_cols:
        mode = tr[col].mode(dropna=True)
        fill_val = mode.iloc[0] if not mode.empty else "unknown"
        tr[col] = tr[col].fillna(fill_val)
        if col in te.columns:
            te[col] = te[col].fillna(fill_val)
    
    tr_enc = pd.get_dummies(tr, drop_first=False)
    te_enc = pd.get_dummies(te, drop_first=False)
    te_enc = te_enc.reindex(columns=tr_enc.columns, fill_value=0)
    
    if y is not None:
        tr_enc[target_col] = y.values
    
    return tr_enc, te_enc

# Uso sugerido (ejecutar si deseas generar conjuntos preparados):
# prepared_train, prepared_test = simple_prepare(train, test)
# display(prepared_train.head())
# display(prepared_test.head())



### Mejores prácticas incorporadas
- **Funciones reutilizables** y normalización de categorías.
- **Comentarios línea a línea** para docencia/entrega.
- **Alineación train/test** tras One-Hot (reindex).
- Cumple todos los puntos: dimensiones, tipos, comparación de columnas, nulos, outliers, distribuciones y relación con `DiagPeriod90`.
